# Memoria EDA - Analisis Precios Supermercados - Esteban Mangado

# Índice

1. [Introducción](#1.-Introducción)  
2. [Obtención de los datos](#2.-Obtención-de-los-datos)
3. [Variables de los datos](#3.-Variables-de-los-datos)
4. [Librerias utilizadas](#4.-Librerias-utilizadas)  
5. [Volumen de trabajo inicial](#5.-Volumen-de-trabajo-inicial)     
6. [Problemas encontrados y soluciones aplicadas](#6.-Problemas-encontrados-y-soluciones-aplicadas)    
7. [Visualización de los resultados](#7.-Visualización-de-los-resultados)   


# 1. Introducción
Este proyecto surge como idea de un comparador de precios entre los distintos supermercados.
Durante este último año han salido muchas noticias derivadas del aumento de los precios en todos los sectores, no solo en el de la alimentación. Por lo que puede ser interesante saber donde se encuentran los precios más caros y donde los más baratos.

Además, gracias a los datos obtenidos, se va a realizar una comparativa entre los precios de Mercadona, Día y Carrefour, estudiando su evolución temporal desde marzo de 2021 hasta el momento de la entrega de este proyecto.

# 2. Obtención de los datos
Los datos se han obtenido gracias a la iniciativa *Open Data de [DataMarket](https://datamarket.es/#productos-de-supermercados-dataset)* para dar acceso a datos actualizados y de calidad de temáticas que son de interés para la comunidad.

Esta iniciativa proporciona datos de tres de los supermercados más importantes del país (Mercadona, Día y Carrefour), con un volumen de datos diarios de aproximadamente unos 50000 registros, que van actualizando en unos CSV descargables desde un [enlace de drive](https://drive.google.com/drive/folders/1nQBn2mkZQyP-pAwD1-gBRVSbGh0unAvr). La serie histórica de datos comienza desde marzo de 2021, lo que permite realizar una comparativa del aumento de los precios en este último año.

Los datos se proporcionan en distintos CSV, separados para cada supermercado y por mes.

# 3. Variables de los datos
Estos datos contienen los siguientes campos (según aparecen inicialmente):

* `url`: URL que lleva a la página del producto. Puede no funcionar si ya no se vende el producto.
* `supermarket`: Supermercado de donde se han obtenido los datos
* `category`: Conjunto del nivel jerárquico de las categorías del producto. Puede contener hasta 3 niveles (en Mercadona el máximo es de dos, Categoría y Subcategoría)
* `name`: Nombre del producto
* `description`: Campo vacío en los datos de Carrefour y Dia. En Mercadona contiene información de si el producto es un paquete, un bote, un pack...
* `price`: Precio que tenía el producto en el momento del registro.
* `reference_price`: Precio de referencia, sobre kilo, litro, unidad...
* `reference_unit`: Unidad del  producto (kg, unidades, litros, lavados...)
* `insert_date`: Fecha en la que se han tomado los datos de los supermercados
* `product_id`: Identificador Id asociado a cada producto

# 4. Librerías utlizadas
Se han utlizado las siguientes librerias
```Python
import pandas as pd
import glob
import numpy as np
from datetime import datetime
from openpyxl import Workbook, load_workbook
import sqlite3
```

* La librería `glob`se ha utilizado para la importación y lectura de los CSV
* `Workbook`y `load_workbook` se emplean para escribir sobre el excel de categorías si fuera necesario
* `datetime`se ha usado para calcular el tiempo que tarda todo el proceso completo en ejecutarse

# 5. Volumen de trabajo inicial

En el momento de realizar este proyecto, se cuenta con el siguiente volumen de información:

* El DataFrame de Carrefour tiene 1967509 de filas y 10 de columnas  
* El DataFrame de Dia tiene 3723032 de filas y 10 de columnas  
* El DataFrame de Mercadona tiene 3855618 de filas y 10 de columnas  

Aunque los dataframe tienen pocas columnas, el gran volumen de datos ha producido varios problemas a lo largo de toda la limpieza de los mismos.

# 6. Problemas encontrados y soluciones aplicadas

Durante este trabajo han aparecido varios problemas.

## Obtención automática de los datos
El plan original era desarrollar un código que descargase automáticamente del Drive los CSV del mes en cuestión. Sin embargo, por la forma de trabajar de Drive, todo lo que se encuentre dentro de una de sus carpetas es considerado como un bloque, no se puede seleccionar de forma trivial los archivos a descargar. Por tanto, la descarga de los datos se realiza manualmente.

## Columna Category
El problema con esta columna era buscar la forma de descomponer el campo category en los diferentes niveles categóricos que emplea cada supermercado para clasificar sus productos.

Sin embargo, debido al gran número de categorías (588 para Carrefour, 769 para Día y 155 para Mercadona) y a que no seguian un patrón para intentar realizarlo con bucles, se tuvo que realizar la categorización manualmente.

Porsteriormetne, se estableció un código que leyese el excel donde está esa categorización y que comprobase si hay categorias nuevas, para en ese caso introducirlas manualmente.

## Outliers
El siguiente gran quebradero de cabeza fueron los outliers de las columnas Price y Reference Price.

Este problema se debía sobre todo a dos cuestiones.
1. Si un producto ha sufrido recientemente una subida, pero durante la mayoria del tiempo ha tenido un precio constante, este último valor aumentado será un Outlier.
2. Si el precio del producto no ha parado de subir, con grandes subidas continuas, los precios más antiguos y los precios más nuevos pueden llegar a ser outliers siguiendo el procedimiento de obtenerlos mediante la desviación típica.

Para solucionar, en cierta medida, este problema se optó por establecer que el valor límite debía ser un múltiplo de la media. Este múltiplo se estableció para cada supermercado y para cada categoria en base a prueba y error.

Esto se debe a que con este método no se captan todos los Outliers que hay, y en cambio también se identifican como Outliers muchos que no lo son, pero en mucha menor medida que cuando se utilizaba el método de la desvicación típica.

## Columna Name
Otro gran problema que ha habido con los datos, es que se ha asignado el mismo "product_id" a productos diferentes.

Esto se debe a que si un producto ha sufrido una modificación en cantidad, por ejemplo pasando de ser un paquete de 500g a uno de 1000g, se ha considerado como el mismo producto, cuando ni por precio ni por nombre lo es.

Aparte de esto, tambien se ha dado el caso de que hay productos iguales (en cantidad y precio similar) que tienen nombres diferentes (con pequeñas variaciones)

Todo esto ha dificultado la unificación de estos nombres. Nombres que son necesarios para buscar por productos en el PowerBI

Para poder unificar los nombres se optó por crear una nueva columna ID que contuviera los "id" y el precio de referencia, agrupando así a los productos.
Una vez agrupados mediante este nuevo campo, se asigna la descripción más actual.

## Tablas de dimensiones y de hechos

El último gran problema encontrado fue una vez terminada la limpieza de los datos, no fue posible cargar los mismos al Power Bi al pesar demasiado las tablas de los DataFrame.

Para solucionar esto, se crearon diversas tablas de dimensiones, aligerando la información de la tabla principal. Esto permitió que se pudieran concatenar los 3 DataFrame en uno solo y meter los más de 9 millones de registros en una única tabla, relacionando el resto de campos (supermercados, fechas, categorías, unidades de referencia...) en otras tablas de dimensiones.

# 7. Visualización de los resultados

Para la visualización de los datos se ha optado por Power BI, creando una pequeña "aplicación" que permite realizar dos funciones principalmente.

La primera, es una comparativa de la evolución de los precios por categorías, analizando los 3 supermercados al mismo tiempo, o cada uno por separado. Para esto se han creado 4 páginas en power bi, una para cada supuesto (conjunto y por separado)

La segunda funcionalidad de la "aplicación" es la de ser un buscador de precios. Este buscador permite comprobar el precio de un producto de un supermercado frente a otros similares de los otros supermercados, viendo la evolución de todos los que se indiquen y comprobando cual es más caro o barato

El enlace del Power BI es el siguiente:

[Enlace](https://app.powerbi.com/reportEmbed?reportId=7dc237a7-855d-4e07-bc74-582495889db5&autoAuth=true&ctid=24a632fe-9754-48c6-b0a6-be004693446d)